# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 21 2023 9:33AM,257816,10,9187447,Emerginnova,Released
1,Feb 21 2023 9:24AM,257811,10,PRF-53716,Bio-PRF,Released
2,Feb 21 2023 9:24AM,257811,10,PRF-53730,Bio-PRF,Released
3,Feb 21 2023 9:24AM,257811,10,PRF-53736,Bio-PRF,Released
4,Feb 21 2023 9:24AM,257811,10,9186931,Bio-PRF,Released
5,Feb 21 2023 9:24AM,257811,10,PRF-53698,Bio-PRF,Released
6,Feb 21 2023 9:24AM,257811,10,PRF-53701,Bio-PRF,Released
7,Feb 21 2023 9:24AM,257811,10,PRF-53704,Bio-PRF,Released
8,Feb 21 2023 9:24AM,257811,10,PRF-53707,Bio-PRF,Released
9,Feb 21 2023 9:24AM,257811,10,PRF-53710,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,257812,Executing,1
37,257813,Released,2
38,257814,Executing,1
39,257815,Released,8
40,257816,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257812,1.0,NaN
257813,NaN,2.0
257814,1.0,NaN
257815,NaN,8.0
257816,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257711,1.0,52.0
257713,0.0,37.0
257715,0.0,56.0
257719,0.0,62.0
257725,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257711,1,52
257713,0,37
257715,0,56
257719,0,62
257725,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257711,1,52
1,257713,0,37
2,257715,0,56
3,257719,0,62
4,257725,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257711,1,52
1,257713,,37
2,257715,,56
3,257719,,62
4,257725,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 21 2023 9:33AM,257816,10,Emerginnova
1,Feb 21 2023 9:24AM,257811,10,Bio-PRF
13,Feb 21 2023 9:24AM,257739,15,"Alliance Pharma, Inc."
32,Feb 21 2023 9:22AM,257815,10,MedStone Pharma LLC
40,Feb 21 2023 9:21AM,257808,10,ISDIN Corporation
80,Feb 21 2023 9:20AM,257814,22,"NBTY Global, Inc."
81,Feb 21 2023 9:20AM,257813,10,"Citieffe, Inc."
83,Feb 21 2023 9:20AM,257812,22,"NBTY Global, Inc."
84,Feb 21 2023 9:19AM,257810,12,Hush Hush
85,Feb 21 2023 9:19AM,257809,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 21 2023 9:33AM,257816,10,Emerginnova,,1
1,Feb 21 2023 9:24AM,257811,10,Bio-PRF,,12
2,Feb 21 2023 9:24AM,257739,15,"Alliance Pharma, Inc.",,19
3,Feb 21 2023 9:22AM,257815,10,MedStone Pharma LLC,,8
4,Feb 21 2023 9:21AM,257808,10,ISDIN Corporation,,40
5,Feb 21 2023 9:20AM,257814,22,"NBTY Global, Inc.",1,
6,Feb 21 2023 9:20AM,257813,10,"Citieffe, Inc.",,2
7,Feb 21 2023 9:20AM,257812,22,"NBTY Global, Inc.",1,
8,Feb 21 2023 9:19AM,257810,12,Hush Hush,,1
9,Feb 21 2023 9:19AM,257809,10,ISDIN Corporation,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 21 2023 9:33AM,257816,10,Emerginnova,1,
1,Feb 21 2023 9:24AM,257811,10,Bio-PRF,12,
2,Feb 21 2023 9:24AM,257739,15,"Alliance Pharma, Inc.",19,
3,Feb 21 2023 9:22AM,257815,10,MedStone Pharma LLC,8,
4,Feb 21 2023 9:21AM,257808,10,ISDIN Corporation,40,
5,Feb 21 2023 9:20AM,257814,22,"NBTY Global, Inc.",,1
6,Feb 21 2023 9:20AM,257813,10,"Citieffe, Inc.",2,
7,Feb 21 2023 9:20AM,257812,22,"NBTY Global, Inc.",,1
8,Feb 21 2023 9:19AM,257810,12,Hush Hush,1,
9,Feb 21 2023 9:19AM,257809,10,ISDIN Corporation,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 21 2023 9:33AM,257816,10,Emerginnova,1,
1,Feb 21 2023 9:24AM,257811,10,Bio-PRF,12,
2,Feb 21 2023 9:24AM,257739,15,"Alliance Pharma, Inc.",19,
3,Feb 21 2023 9:22AM,257815,10,MedStone Pharma LLC,8,
4,Feb 21 2023 9:21AM,257808,10,ISDIN Corporation,40,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 21 2023 9:33AM,257816,10,Emerginnova,1.0,NaN
1,Feb 21 2023 9:24AM,257811,10,Bio-PRF,12.0,NaN
2,Feb 21 2023 9:24AM,257739,15,"Alliance Pharma, Inc.",19.0,NaN
3,Feb 21 2023 9:22AM,257815,10,MedStone Pharma LLC,8.0,NaN
4,Feb 21 2023 9:21AM,257808,10,ISDIN Corporation,40.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 21 2023 9:33AM,257816,10,Emerginnova,1.0,0.0
1,Feb 21 2023 9:24AM,257811,10,Bio-PRF,12.0,0.0
2,Feb 21 2023 9:24AM,257739,15,"Alliance Pharma, Inc.",19.0,0.0
3,Feb 21 2023 9:22AM,257815,10,MedStone Pharma LLC,8.0,0.0
4,Feb 21 2023 9:21AM,257808,10,ISDIN Corporation,40.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6959936,553.0,9.0
12,515614,39.0,0.0
15,515474,41.0,0.0
16,257742,0.0,1.0
19,515595,13.0,10.0
22,515626,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6959936,553.0,9.0
1,12,515614,39.0,0.0
2,15,515474,41.0,0.0
3,16,257742,0.0,1.0
4,19,515595,13.0,10.0
5,22,515626,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,553.0,9.0
1,12,39.0,0.0
2,15,41.0,0.0
3,16,0.0,1.0
4,19,13.0,10.0
5,22,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,553.0
1,12,Released,39.0
2,15,Released,41.0
3,16,Released,0.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Executing,9.0,0.0,0.0,1.0,10.0,2.0
Released,553.0,39.0,41.0,0.0,13.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Executing,9.0,0.0,0.0,1.0,10.0,2.0
1,Released,553.0,39.0,41.0,0.0,13.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Executing,9.0,0.0,0.0,1.0,10.0,2.0
1,Released,553.0,39.0,41.0,0.0,13.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()